## Attractor Landscape analysis

This is an example of how Boolean network model was analyzed in our paper.

We provide the codes for 'Attractor landscape analysis', 'Perturbation analysis', and 'Analysis of network dynamics' (Please refer to the Method section).


In [1]:
import numpy as np
import pandas as pd
import itertools
import networkx as nx
import copy
import os

from pyboolnet.file_exchange import bnet2primes, primes2bnet
from pyboolnet.interaction_graphs import primes2igraph
from pyboolnet.state_transition_graphs import primes2stg
from pyboolnet.attractors import compute_attractors_tarjan

from modules.attractorSim import rand_initial_states, compute_attractor_from_primes, compute_phenotype, Simulation
from modules.stability import compute_networkStability

In [2]:
network_dir = './network/'
model_file = network_dir + 'toy_Network.bnet'
primes = bnet2primes(model_file)
nodeList = list(primes.keys())
graph = primes2igraph(primes)
update_mode = "synchronous"

phenotype = {'P1':{'x4':1,'x5':0}, 'P2':{'x4':0, 'x5':1}, 'P3':{'x4':1, 'x5':1}, 'P4':{'x4':0, 'x5':0}}
phenotypeAnnot = {'P1':-1,'P2':1, 'P3':0, 'P4':0}
markers = ['x4','x5']


if 2**len(nodeList) >= 100000: num_init = 100000
else:  num_init = 2**len(nodeList)
initState = rand_initial_states(num_init, len(nodeList))

### Node perturbation analysis

In [3]:
save_dir = './result/toy/' 
save_perturbname = save_dir + 'toy_single_simul_result.csv'

In [4]:
fix_dict = {'x1':0}
allsingles = [{n:1} for n in nodeList] + [{n:0} for n in nodeList]
#allsingles

In [5]:
perturb_p = pd.DataFrame([]) # average activities of the marker nodes
perturb_s = pd.DataFrame([]) # network stabiltiy 
for perturb in allsingles:
    fix_dict_tmp = copy.deepcopy(fix_dict)
    fix_dict_tmp.update(perturb)
    print(fix_dict_tmp)
    primes_new, pheno_df, att_ave_pd, attrs_dict = Simulation(fix_dict_tmp, primes, update_mode, initState, phenotype, phenotypeAnnot)
    dT = pd.DataFrame([0 for _ in markers], index = markers, columns = [str(list(perturb.items()))])
    dT.loc[markers,:] = att_ave_pd.loc[markers,:].values
    perturb_p = pd.concat([perturb_p,dT],axis=1)     
    dS = pd.DataFrame.from_dict(compute_networkStability(attrs_dict, graph, nodeList)[1], orient='index', columns = [str(list(perturb.items()))] )
    perturb_s = pd.concat([perturb_s,dS],axis=1)    
perturb_result = pd.concat([perturb_p, perturb_s]).T        
perturb_result.to_csv(save_perturbname)

{'x1': 1}
Attractor simulation time : 0.05311012268066406
            Ratio
phenotype        
P1         0.1875
P3         0.8125
{'x1': 0, 'x2': 1}
Attractor simulation time : 0.025899171829223633
            Ratio
phenotype        
P1         0.1875
P3         0.8125
{'x1': 0, 'x3': 1}
Attractor simulation time : 0.03022480010986328
           Ratio
phenotype       
P2           1.0
{'x1': 0, 'x4': 1}
Attractor simulation time : 0.03718137741088867
            Ratio
phenotype        
P1         0.1875
P3         0.8125
{'x1': 0, 'x5': 1}
Attractor simulation time : 0.031427860260009766
           Ratio
phenotype       
P2           1.0
{'x1': 0, 'x6': 1}
Attractor simulation time : 0.03730010986328125
           Ratio
phenotype       
P2           1.0
{'x1': 0, 'x7': 1}
Attractor simulation time : 0.02576899528503418
              Ratio
phenotype          
P1         0.187500
P2         0.359375
P3         0.453125
{'x1': 0, 'x8': 1}
Attractor simulation time : 0.02559685707092285
  

In [6]:
perturb_p # average activities of the marker nodes

,"[('x1', 1)]","[('x2', 1)]","[('x3', 1)]","[('x4', 1)]","[('x5', 1)]","[('x6', 1)]","[('x7', 1)]","[('x8', 1)]","[('x9', 1)]","[('x1', 0)]","[('x2', 0)]","[('x3', 0)]","[('x4', 0)]","[('x5', 0)]","[('x6', 0)]","[('x7', 0)]","[('x8', 0)]","[('x9', 0)]"
x4,1.000000,1.000000,0.0,1.0000,0.0,0.0,0.414062,0.414062,0.414062,0.414062,0.414062,1.0000,0.0000,1.0,1.0,0.414062,0.414062,0.414062
x5,0.585938,0.585938,1.0,0.5625,1.0,1.0,0.585938,0.585938,0.585938,0.585938,0.585938,0.5625,0.8125,0.0,0.0,0.585938,0.585938,0.585938


### Frustration of a steady state

In [9]:
perturb_s.loc['F_major_att',:] # frustration of a major attractor

[('x1', 1)]   -11.0
[('x2', 1)]    -3.0
[('x3', 1)]   -11.0
[('x4', 1)]     5.0
[('x5', 1)]    -9.0
[('x6', 1)]   -11.0
[('x7', 1)]     7.0
[('x8', 1)]     5.0
[('x9', 1)]     1.0
[('x1', 0)]     6.0
[('x2', 0)]     6.0
[('x3', 0)]    -1.0
[('x4', 0)]    -9.0
[('x5', 0)]    -7.0
[('x6', 0)]    -7.0
[('x7', 0)]     5.0
[('x8', 0)]     1.0
[('x9', 0)]    -3.0
Name: F_major_att, dtype: float64